<a href="https://colab.research.google.com/github/braveenth/Canadian-Weather-AI/blob/main/Canadian_Weather_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Canadian Weather AI
By: Braveenth Rasanayagam

Multi-modal
- Vision
- Text to Voice

Cloud Run Jobs

In [1]:
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
print(image_url)
!curl -o case_image.png {image_url}

from IPython.display import Image
Image("/content/jet_stream.png")
Image(url=image_url)

https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  268k  100  268k    0     0  1010k      0 --:--:-- --:--:-- --:--:-- 1011k


In [2]:
import os
import google.generativeai as genai

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    !pip install openai
    !pip install pydub
    !pip install anthropic
    !pip install -U -q google-generativeai
    !pip install langchain_groq
    !pip install groq
    from openai import OpenAI
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHRONPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('AI_STUDIO_KEY')
    HF_API_KEY = userdata.get('HF_ACCESS_KEY')
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    !mkdir -p /content/text
    !mkdir -p /content/voice
    !mkdir -p /content/assets
    !mkdir -p /content/metadata
    !mkdir -p /content/weatherperson
    !curl -o "/content/assets/WeatherNetwork.mp3" "https://storage.googleapis.com/can-weather-ai/assets/WeatherNetwork.mp3"

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    print("Running outside of Google Colab")
    from openai import OpenAI
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
    ANTHRONPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = os.environ.get('GOOGLE_AI_STUDIO_KEY')
    HF_API_KEY = os.environ.get('HF_ACCESS_KEY')
    GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

Running on Google Colab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.8/870.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
 

In [3]:
# This will be incorporated into the prompt in the future

canadianMetric = {}

canadianMetric[0] = "not Canadian"
canadianMetric[1] = "somewhat Canadian"
canadianMetric[2] = "slightly Canadian"
canadianMetric[3] = "moderately Canadian"
canadianMetric[4] = "very Canadian"
canadianMetric[5] = "as Canadian as possible"

selectedCanadianMetric = canadianMetric[4]
langauge = "en"

# We are allowing for multiple models to be incorporated
modelChoice = {}

modelChoice[0] = "gpt-4-vision-preview"
modelChoice[1] = "claude-3-opus-20240229"
modelChoice[2] = "gemini-pro-vision"
modelChoice[3] = "gemini-1.5-pro-latest"
modelChoice[4] = "llama3-70b-8192"
modelChoice[5] = "gpt-4o"

selectedModel = modelChoice[5]

In [5]:
makeLyrics = False
musicPrompt = {}
musicPrompt[0] = "Make lyrics for a beautiful melodic song."
musicPrompt[1] = "Make lyrics for a alternative rock song, male vocals, powerful."
musicPrompt[2] = "Make lyrics for a RnB song, female vocals, 2000s influence."


In [7]:
imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"

if selectedModel[0:5] == "gpt-4":
  gpt_mode = selectedModel
  imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
  image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
  multimodalModelOrg = "OpenAI"

  response = client.chat.completions.create(
    model=gpt_mode,
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": imagePrompt},
          {
            "type": "image_url",
            "image_url": {
              "url": image_url,
            },
          },
        ],
      }
    ],
    max_tokens=600,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  weatherScript = response.choices[0].message.content
  print(response.choices[0])

elif selectedModel == "claude-3-opus-20240229":
  import anthropic
  import base64
  import httpx
  claude_mode = "claude-3-opus-20240229"
  multimodalModelOrg = "Anthropic"

  client = anthropic.Anthropic(
      api_key=ANTHRONPIC_API_KEY,
  )

  image_media_type = "image/gif"
  image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
  imagePrompt += "Do not put any statements with asterisks surrounding them. Only give the speech, and nothing else. For example, statements similar to the following should be excluded: *clears throat*"

  message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1024,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "image",
                      "source": {
                          "type": "base64",
                          "media_type": image_media_type,
                          "data": image_data,
                      },
                  },
                  {
                      "type": "text",
                      "text": imagePrompt
                  }
              ],
          }
      ],
  )
  weatherScript = message.content[0].text
  print(weatherScript)


elif selectedModel == "gemini-1.5-pro-latest":
  # from vertexai.generative_models import GenerativeModel, Part, FinishReason
  # import vertexai.preview.generative_models as generative_models
  # model = genai.GenerativeModel('models/gemini-1.5-pro-latest')
  #model = genai.GenerativeModel('models/gemini-1.5-pro-latest')
  #imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
  imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
  image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
  multimodalModelOrg = "Google"

  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
  }

  safety_settings = {
    # generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    # generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    # generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    # generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
  }

  model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                generation_config=generation_config,
                                safety_settings=safety_settings)

  # print(.GenerativeModel.ListModels())
  response = model.generate_content(
      [imagePrompt, image_url],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
      )

  # for response in responses:
  #   print(response.text, end="")

  weatherScript = response.text
  print(weatherScript)

elif selectedModel == "llama3-70b-8192-GPU":
  # from langchain_groq import Groq
  import transformers
  import torch
  from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
  model_id = "meta-llama/Meta-Llama-3-8B"
  !pip install accelerate

  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", token=HF_API_KEY)
  model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", token=HF_API_KEY)

elif selectedModel == "llama3-70b-8192":
  prompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible. You will only output what the weatherperson would speak."

  if makeLyrics == True:
    prompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. You will only output lyrics. Make sure you cover the weather information so that listeners are aware of the resulting weather from the jet stream information."
    prompt += musicPrompt[0]
    prompt += "Use [Verse], [Bridge], and [Chorus] tags for additional control."

  multimodalModelOrg = "Meta"
  from groq import Groq

  client = Groq(
      api_key=GROQ_API_KEY,
  )

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": f"Image: {image_url}. {prompt}",
          }
      ],
      model="llama3-70b-8192",
  )

  client.chat.completions.create

  weatherScript = chat_completion.choices[0].message.content


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Oh, hey there, Canada! Got a beauty of a jet stream update for ya, so grab your Timmies and let's dive in, eh?\n\nAlrighty, folks, up in the north, we've got some chilly temperatures, as you'd expect this time of year. The jet stream is dipping down, bringing some of that cold Arctic air with it. We’re seeing temperatures in the minus 30s up there in the territories. So bundle up if you're heading out, and don’t forget your toque!\n\nMoving west, over B.C., we've got a low pressure system hanging around. It's pushing warmer air from the Pacific inland, so parts of the province are seeing temperatures above 10 degrees. Not too shabby, eh? But watch out for some wet and wild weather as that moisture moves in.\n\nNow, over the Prairies, it's a mixed bag, folks. The jet stream is pretty active here, with a bit of everything going on. We've got some warmer air trying to sneak in from the south, but t

In [8]:
print(weatherScript)

Oh, hey there, Canada! Got a beauty of a jet stream update for ya, so grab your Timmies and let's dive in, eh?

Alrighty, folks, up in the north, we've got some chilly temperatures, as you'd expect this time of year. The jet stream is dipping down, bringing some of that cold Arctic air with it. We’re seeing temperatures in the minus 30s up there in the territories. So bundle up if you're heading out, and don’t forget your toque!

Moving west, over B.C., we've got a low pressure system hanging around. It's pushing warmer air from the Pacific inland, so parts of the province are seeing temperatures above 10 degrees. Not too shabby, eh? But watch out for some wet and wild weather as that moisture moves in.

Now, over the Prairies, it's a mixed bag, folks. The jet stream is pretty active here, with a bit of everything going on. We've got some warmer air trying to sneak in from the south, but those northern winds are keeping things cool. So, if you're out in Alberta, Saskatchewan, or Manito

In [9]:
# Save the weatherScript as a file
from datetime import datetime

# Adding timezone
import pytz
timezone = pytz.timezone('America/Toronto')
now_in_timezone = datetime.now(timezone)

# Format the date
date = now_in_timezone.strftime("%Y-%m-%d")

#date = datetime.now().strftime("%Y-%m-%d")
count = 0

while os.path.exists(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt"):
    count += 1
file = open(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt", "w")
file.write(weatherScript)
file.close()

In [ ]:
from openai import OpenAI

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    audioClient = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    import os
    print("Running outside of Google Colab")
    audioClient = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

audioModelOrg = "OpenAI"
audioModel = "tts-1"
audioVoice = "onyx"

audioResponse = audioClient.audio.speech.create(
    model=audioModel,
    voice=audioVoice,
    input=weatherScript,
)

filename = f"weatherperson-output-{date}.mp3"

Running on Google Colab


In [ ]:
audioResponse.stream_to_file(filename)

<ipython-input-25-02f298be9c94>:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(filename)


In [ ]:
from pydub import AudioSegment

# Export the result
weatherPersonFileCount = 0
while os.path.exists(f"/content/weatherperson/weatherperson-output-{date}_{weatherPersonFileCount}.wav"):
    weatherPersonFileCount += 1

weatherpersonFile = f"/content/weatherperson/{filename}-{count}"
audioResponse.stream_to_file(f"/content/weatherperson/weatherperson-output-{date}_{weatherPersonFileCount}.wav")

# Obtain Audio Files
audio_bg_music = AudioSegment.from_file("/content/assets/WeatherNetwork.mp3", format="mp3")
audio_weatherperson = AudioSegment.from_file(f"/content/weatherperson/weatherperson-output-{date}_{weatherPersonFileCount}.wav", format="mp3")

# BG Music Segments from https://www.youtube.com/watch?v=6E2uzYAGPQU
#####################################################################
# 01. Morning Report (1998) Conclusions 0:00. #
# 02. Autumn (1999) Flying V.1 1:59 #
# 03. Holidays (2000) Xmas Spirit 3:29
# 04. Biding My Time 4:59
# 05. Winter (2002-'03) Life to the Full 7:12
# 06. Spring-Autumn (2003) Healthy Outlook 8:42
# 07. Winter (2003-04) Embrace Life 10:43
# 08. Primary Theme (2004-05) Lazing on the Slopes 13:16
# 09. Theme (2005-06) Windstar 15:17
# 10. Theme (2006-10) 17:17
# 11. Theme (2010-201?) Song Contest Winner 19:37
# 12. Holiday Version of Theme 11 (2011) 21:58
# 13. 25th Anniversary (2004-05 Secondary Theme) 25:02

segments = [0] * 12
segments[0] = [0, 119000, 7]
segments[1] = [119000, 209000, 7]
segments[2] = [209000, 299000, 7]
segments[3] = [299000, 432000, 7]
segments[4] = [432000, 522000, 7]
segments[5] = [522000, 643000, 7]
segments[6] = [643000, 796000, 7]
segments[7] = [796000, 917000, 7]
segments[8] = [917000, 1037000, 7]
segments[9] = [1037000, 1177000, 7]
segments[10] = [1177000, 1318000, 11]
segments[11] = [1318000, 1502000, 7]

# Custom timing adjustments based on listening observations
segments[8][0] = segments[8][0] + 8000

# Define start and end times in milliseconds
# Start at 17:22
# End at 17:3
start_time = 1042000  # Start at 10 seconds
end_time = start_time + 120000

# Use a Random number between 0 an 11 to obtain the segment number
import random
segment_number = random.randint(0, 11)

# Using Segments Here
start_time = segments[segment_number][0] + 1000
end_time = segments[segment_number][1]

# Slice the audio segment to the desired part
specific_part = audio_bg_music[start_time:end_time]

# Lower the volume by 10 dB
specific_part_quieter = specific_part - segments[segment_number][2]

# TO-DO: if the audio_weatherperson audio length is longer than the specific_part_quieter, then add some padding to the end of the specific_part_quieter
lenBackgroundMusicMS = end_time - start_time
lenAudioWeatherPersonMS = len(audio_weatherperson)
if lenAudioWeatherPersonMS > lenBackgroundMusicMS:
    from pydub.generators import Silence
    padding_length_ms = lenAudioWeatherPersonMS - lenBackgroundMusicMS + 1000
    #padding = Silence().to_audio_segment(duration=padding_length_ms, frame_rate=44100)
    padding = AudioSegment.silent(duration=padding_length_ms)
    specific_part_quieter = specific_part_quieter.append(padding)

# TO_DO: Add audio ducking

# Overlay the audio files
# Here, overlay_audio will start at 0 milliseconds into base_audio
combined = specific_part_quieter.overlay(audio_weatherperson, position=0)

# Export the result
count = 0
while os.path.exists(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3"):
    count += 1
combined.export(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3", format="mp3")


<ipython-input-26-0b25720dc80f>:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(f"/content/weatherperson/weatherperson-output-{date}_{weatherPersonFileCount}.wav")


<_io.BufferedRandom name='/content/voice/ai-canadian-jetstream-2024-05-01_0.mp3'>

In [ ]:
# Escape the newlines in the weatherScript
weatherScriptEscapedString = weatherScript.replace("\n", "\\n")
print(weatherScriptEscapedString)

"Buddy, take a gander at this jet stream map, eh! You'll notice we've got a bit of a wild ride goin' on across the Great White North, ay. A strong high-pressure system is parked over the Prairie provinces, bringin' some nice, warm air from the southwest, and that's gonna give us a real nice stretch of weather, especially in Alberta and Saskatchewan. You folks in Calgary and Edmonton, get ready for a beaut day, with temps soaring into the mid-teens, Celsius, of course! That's practically shorts weather, buddy!\n\nNow, on the flip side, we've got a low-pressure system movin' into the Maritimes, and that's gonna bring some wet and windy weather to our friends in Nova Scotia, New Brunswick, and Prince Edward Island. Sorry, buddies, but it looks like the umbrella's gonna be your best friend for the next 24 hours.\n\nMeanwhile, out in British Columbia, you're lookin' at some lovely, mild air comin' in off the Pacific, so Vancouver and Victoria, you're in for a treat, eh! Just a beautiful day

In [ ]:
# TO-DO: Create JSON that can be imported into BigQuery
# Use Colab's Variable Inspector to determine which variables shoudl be captured.

metadata_json = f"""{{
  "timestamp": "{date}",
  "timezone": "{timezone}",
  "weatherScript": "{weatherScriptEscapedString}",
  "weatherScriptPath": "/content/text/ai-canadian-jetstream-{date}_{count}.txt",
  "modelOrganization": "{multimodalModelOrg}",
  "modelName": "{selectedModel}",
  "imagePrompt": "{imagePrompt}",
  "imageUrl": "{image_url}",
  "audioModelOrganization": "{audioModelOrg}",
  "audioModelName": "{audioModel}",
  "audioVoice": "{audioVoice}",
  "speechAudioOutputPath": "/content/weatherperson/weatherperson-output-{date}_{weatherPersonFileCount}.wav",
  "combinedAudioOutputPath": "/content/voice/ai-canadian-jetstream-{date}_{count}.mp3"
}}
"""

In [ ]:
print(metadata_json)

{
  "timestamp": "2024-05-01",
  "timezone": "America/Toronto",
  "weatherScript": ""Buddy, take a gander at this jet stream map, eh! You'll notice we've got a bit of a wild ride goin' on across the Great White North, ay. A strong high-pressure system is parked over the Prairie provinces, bringin' some nice, warm air from the southwest, and that's gonna give us a real nice stretch of weather, especially in Alberta and Saskatchewan. You folks in Calgary and Edmonton, get ready for a beaut day, with temps soaring into the mid-teens, Celsius, of course! That's practically shorts weather, buddy!\n\nNow, on the flip side, we've got a low-pressure system movin' into the Maritimes, and that's gonna bring some wet and windy weather to our friends in Nova Scotia, New Brunswick, and Prince Edward Island. Sorry, buddies, but it looks like the umbrella's gonna be your best friend for the next 24 hours.\n\nMeanwhile, out in British Columbia, you're lookin' at some lovely, mild air comin' in off the

In [ ]:
# Write the metadata file
count = 0

while os.path.exists(f"/content/metadata/ai-canadian-jetstream-metadata-{date}_{count}.json"):
    count += 1
file = open(f"/content/metadata/ai-canadian-jetstream-metadata-{date}_{count}.json", "w")
file.write(metadata_json)
file.close()